In [101]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

from statsmodels.tsa.seasonal import STL


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Step 1. Data Import & Clean Up

In [102]:
df = pd.read_csv("kaggle/input/2019-2024-us-stock-market-data/Stock Market Dataset.csv")

In [103]:
df.head(5)

,Unnamed: 0,Date,Natural_Gas_Price,Natural_Gas_Vol.,Crude_oil_Price,Crude_oil_Vol.,Copper_Price,Copper_Vol.,Bitcoin_Price,Bitcoin_Vol.,Platinum_Price,Platinum_Vol.,Ethereum_Price,Ethereum_Vol.,S&P_500_Price,Nasdaq_100_Price,Nasdaq_100_Vol.,Apple_Price,Apple_Vol.,Tesla_Price,Tesla_Vol.,Microsoft_Price,Microsoft_Vol.,Silver_Price,Silver_Vol.,Google_Price,Google_Vol.,Nvidia_Price,Nvidia_Vol.,Berkshire_Price,Berkshire_Vol.,Netflix_Price,Netflix_Vol.,Amazon_Price,Amazon_Vol.,Meta_Price,Meta_Vol.,Gold_Price,Gold_Vol.
0,0,02-02-2024,2.079,NaN,72.28,NaN,3.8215,NaN,"43,194.70",42650.0,901.6,NaN,"2,309.28",246890.0,"4,958.61","17,642.73",315620000.0,185.85,102550000.0,187.91,110610000.0,411.22,28260000.0,22.796,NaN,142.38,62500000.0,661.60,47660000.0,"5,89,498",10580.0,564.64,4030000.0,171.81,117220000.0,474.99,84710000.0,"2,053.70",NaN
1,1,01-02-2024,2.050,161340.0,73.82,577940.0,3.8535,NaN,"43,081.40",47690.0,922.3,NaN,"2,304.28",323610.0,"4,906.19","17,344.71",240640000.0,186.86,53490000.0,188.86,90680000.0,403.78,29230000.0,23.236,85160.0,141.16,37120000.0,630.27,36020000.0,"5,81,600",9780.0,567.51,3150000.0,159.28,66360000.0,394.78,25140000.0,"2,071.10",260920.0
2,2,31-01-2024,2.100,142860.0,75.85,344490.0,3.9060,NaN,"42,580.50",56480.0,932.6,NaN,"2,283.14",408790.0,"4,848.87","17,137.24",366450000.0,184.40,54830000.0,187.29,102270000.0,397.58,46780000.0,23.169,66910.0,140.10,71370000.0,615.27,45070000.0,"5,78,020",9720.0,564.11,4830000.0,155.20,49690000.0,390.14,20010000.0,"2,067.40",238370.0
3,3,30-01-2024,2.077,139750.0,77.82,347240.0,3.9110,NaN,"42,946.20",55130.0,931.7,NaN,"2,343.11",387120.0,"4,924.97","17,476.71",236210000.0,188.04,55270000.0,191.59,105540000.0,408.59,29340000.0,23.225,53370.0,151.46,33060000.0,627.74,39600000.0,"5,84,680",9750.0,562.85,6120000.0,159.00,42290000.0,400.06,18610000.0,"2,050.90",214590.0
4,4,29-01-2024,2.490,3590.0,76.78,331930.0,3.8790,NaN,"43,299.80",45230.0,938.3,NaN,"2,317.79",318840.0,"4,927.93","17,596.27",238750000.0,191.73,46890000.0,190.93,123600000.0,409.72,23290000.0,23.134,330.0,153.51,27590000.0,624.65,33900000.0,"5,78,800",13850.0,575.79,6880000.0,161.26,42840000.0,401.02,17790000.0,"2,034.90",1780.0


In [104]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1243 entries, 0 to 1242
Data columns (total 39 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         1243 non-null   int64  
 1   Date               1243 non-null   object 
 2   Natural_Gas_Price  1243 non-null   float64
 3   Natural_Gas_Vol.   1239 non-null   float64
 4   Crude_oil_Price    1243 non-null   float64
 5   Crude_oil_Vol.     1220 non-null   float64
 6   Copper_Price       1243 non-null   float64
 7   Copper_Vol.        1206 non-null   float64
 8   Bitcoin_Price      1243 non-null   object 
 9   Bitcoin_Vol.       1243 non-null   float64
 10  Platinum_Price     1243 non-null   object 
 11  Platinum_Vol.      636 non-null    float64
 12  Ethereum_Price     1243 non-null   object 
 13  Ethereum_Vol.      1243 non-null   float64
 14  S&P_500_Price      1243 non-null   object 
 15  Nasdaq_100_Price   1243 non-null   object 
 16  Nasdaq_100_Vol.    1242 

### Check the formatting of those non-numeric columns

In [105]:
df[['Date', 'Bitcoin_Price', 'Platinum_Price', 'Ethereum_Price', 'S&P_500_Price', 'Nasdaq_100_Price', 'Berkshire_Price', 'Gold_Price']].head(5)

,Date,Bitcoin_Price,Platinum_Price,Ethereum_Price,S&P_500_Price,Nasdaq_100_Price,Berkshire_Price,Gold_Price
0,02-02-2024,"43,194.70",901.6,"2,309.28","4,958.61","17,642.73","5,89,498","2,053.70"
1,01-02-2024,"43,081.40",922.3,"2,304.28","4,906.19","17,344.71","5,81,600","2,071.10"
2,31-01-2024,"42,580.50",932.6,"2,283.14","4,848.87","17,137.24","5,78,020","2,067.40"
3,30-01-2024,"42,946.20",931.7,"2,343.11","4,924.97","17,476.71","5,84,680","2,050.90"
4,29-01-2024,"43,299.80",938.3,"2,317.79","4,927.93","17,596.27","5,78,800","2,034.90"


In [106]:
for col in ['Bitcoin_Price', 'Platinum_Price', 'Ethereum_Price', 'S&P_500_Price', 'Nasdaq_100_Price', 'Berkshire_Price', 'Gold_Price']:
    
    df[col] = df[col].str.replace(',','')
    
df = df.astype({'Bitcoin_Price': 'float64', 'Platinum_Price': 'float64', 'Ethereum_Price': 'float64', 
                'S&P_500_Price': 'float64', 'Nasdaq_100_Price': 'float64', 'Berkshire_Price': 'float64', 'Gold_Price': 'float64'})

In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1243 entries, 0 to 1242
Data columns (total 39 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         1243 non-null   int64  
 1   Date               1243 non-null   object 
 2   Natural_Gas_Price  1243 non-null   float64
 3   Natural_Gas_Vol.   1239 non-null   float64
 4   Crude_oil_Price    1243 non-null   float64
 5   Crude_oil_Vol.     1220 non-null   float64
 6   Copper_Price       1243 non-null   float64
 7   Copper_Vol.        1206 non-null   float64
 8   Bitcoin_Price      1243 non-null   float64
 9   Bitcoin_Vol.       1243 non-null   float64
 10  Platinum_Price     1243 non-null   float64
 11  Platinum_Vol.      636 non-null    float64
 12  Ethereum_Price     1243 non-null   float64
 13  Ethereum_Vol.      1243 non-null   float64
 14  S&P_500_Price      1243 non-null   float64
 15  Nasdaq_100_Price   1243 non-null   float64
 16  Nasdaq_100_Vol.    1242 

## Data Grouping

In [108]:
df.columns

Index(['Unnamed: 0', 'Date', 'Natural_Gas_Price', 'Natural_Gas_Vol.',
       'Crude_oil_Price', 'Crude_oil_Vol.', 'Copper_Price', 'Copper_Vol.',
       'Bitcoin_Price', 'Bitcoin_Vol.', 'Platinum_Price', 'Platinum_Vol.',
       'Ethereum_Price', 'Ethereum_Vol.', 'S&P_500_Price', 'Nasdaq_100_Price',
       'Nasdaq_100_Vol.', 'Apple_Price', 'Apple_Vol.', 'Tesla_Price',
       'Tesla_Vol.', 'Microsoft_Price', 'Microsoft_Vol.', 'Silver_Price',
       'Silver_Vol.', 'Google_Price', 'Google_Vol.', 'Nvidia_Price',
       'Nvidia_Vol.', 'Berkshire_Price', 'Berkshire_Vol.', 'Netflix_Price',
       'Netflix_Vol.', 'Amazon_Price', 'Amazon_Vol.', 'Meta_Price',
       'Meta_Vol.', 'Gold_Price', 'Gold_Vol.'],
      dtype='object')

In [109]:
_commodity_price = ['Natural_Gas_Price', 'Crude_oil_Price', 'Copper_Price', 'Platinum_Price', 'Silver_Price', 'Gold_Price']
_commodity_vol   = ['Natural_Gas_Vol.', 'Crude_oil_Vol.', 'Copper_Vol.', 'Platinum_Vol.', 'Silver_Vol.', 'Gold_Vol.']
_crypto_price    = ['Bitcoin_Price', 'Ethereum_Price']
_crypto_vol      = ['Bitcoin_Vol.', 'Ethereum_Vol.']
_index_price     = ['S&P_500_Price', 'Nasdaq_100_Price', 'Berkshire_Price']
_index_vol       = ['Nasdaq_100_Vol.', 'Berkshire_Vol.']
_techs_price     = ['Apple_Price', 'Tesla_Price', 'Microsoft_Price', 'Google_Price', 'Nvidia_Price', 'Netflix_Price', 'Amazon_Price', 'Meta_Price']
_techs_vol       = ['Apple_Vol.', 'Tesla_Vol.', 'Microsoft_Vol.', 'Google_Vol.', 'Nvidia_Vol.', 'Netflix_Vol.', 'Amazon_Vol.', 'Meta_Vol.']

### `Date`

In [110]:
df["Date"] = pd.to_datetime(df["Date"], format='%d-%m-%Y')

In [111]:
df = df.set_index("Date")\
    .drop(columns=df.columns[0])\
    .sort_index()

### `Commodity`

In [112]:
df[_commodity_price + _commodity_vol].describe(include = 'all')

,Natural_Gas_Price,Crude_oil_Price,Copper_Price,Platinum_Price,Silver_Price,Gold_Price,Natural_Gas_Vol.,Crude_oil_Vol.,Copper_Vol.,Platinum_Vol.,Silver_Vol.,Gold_Vol.
count,1243.000000,1243.000000,1243.000000,1243.000000,1243.000000,1243.000000,1239.000000,1.220000e+03,1206.000000,636.000000,1196.000000,1241.000000
mean,3.494714,67.577064,3.541957,959.003620,21.588977,1759.246742,131624.116223,3.989038e+05,35406.616915,9082.515723,67695.418060,211127.671233
std,1.822540,20.465500,0.702819,108.012849,3.859288,203.258901,64385.141749,2.161619e+05,38415.448731,8876.538587,38078.851679,115006.351292
min,1.482000,-37.630000,2.100500,595.200000,11.772000,1272.000000,1200.000000,1.702000e+04,10.000000,0.000000,0.000000,0.000000
25%,2.347500,55.095000,2.858750,889.775000,17.998500,1669.600000,91900.000000,2.835975e+05,370.000000,1120.000000,48150.000000,152200.000000
50%,2.702000,69.230000,3.666000,944.700000,22.758000,1804.200000,127370.000000,3.668850e+05,10180.000000,6070.000000,62940.000000,197970.000000
75%,4.055500,80.455000,4.137250,1020.400000,24.512000,1912.800000,169460.000000,5.072425e+05,68340.000000,15287.500000,79100.000000,257920.000000
max,9.647000,123.700000,4.937500,1297.100000,29.418000,2089.700000,381970.000000,1.770000e+06,176040.000000,42830.000000,355280.000000,813410.000000


In [113]:
fig = px.line(df, y=_commodity_price, title='Commodity Price Time Series', width = 1000, height = 400)

fig.update_xaxes(
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=3, label="3m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

In [114]:
fig = px.line(df, y=_commodity_vol, title='Commodity Vol. Time Series', width = 1000, height = 400)

fig.update_xaxes(
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=3, label="3m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

In [115]:
df[_commodity_vol].describe().loc['count'] / len(df)

Natural_Gas_Vol.    0.996782
Crude_oil_Vol.      0.981496
Copper_Vol.         0.970233
Platinum_Vol.       0.511665
Silver_Vol.         0.962188
Gold_Vol.           0.998391
Name: count, dtype: float64

- The missing values in commodities trade volume couldn't be easily fixed by mean/zero imputation as the trade volume is highly volatile.
- Linear interpolation could be a quick fix for Natural Gas and Gold which has less than 1% missing trade volume data.
- Interpolation with seasonal decomposition adjustment could be useful to Crude Oil, Copper, and Silver as they have chuncks of missing data. But for simplicity linear interpolation is good enough.
- Platinum Volume has 49% missing data, also the trend shows a very clear periodic seasonal factor and mean imputation / linear interpolcation is not applicable.There are no choice but dropping the column.

In [116]:
df[_commodity_vol] = df[_commodity_vol].interpolate(method='linear', limit_direction='both')
df.drop(columns = 'Platinum_Vol.', inplace=True)
_commodity_vol  = ['Natural_Gas_Vol.', 'Crude_oil_Vol.', 'Copper_Vol.', 'Silver_Vol.', 'Gold_Vol.']



### `Crypto`

In [117]:
df[_crypto_price + _crypto_vol].describe()

,Bitcoin_Price,Ethereum_Price,Bitcoin_Vol.,Ethereum_Vol.
count,1243.000000,1243.000000,1.243000e+03,1.243000e+03
mean,25241.903057,1445.815133,4.033918e+07,1.801563e+07
std,16029.009055,1160.674814,2.940889e+08,1.326933e+08
min,3397.700000,104.550000,2.600000e+02,7.518000e+04
25%,10014.600000,244.080000,7.907500e+04,5.883600e+05
50%,23055.100000,1552.600000,2.153100e+05,1.570000e+06
75%,37784.200000,2056.555000,6.151050e+05,9.365000e+06
max,67527.900000,4808.380000,4.470000e+09,1.790000e+09


In [118]:
fig = px.line(df, y=_crypto_price, title='Crypto Price Time Series', width = 1000, height = 400)

fig.update_xaxes(
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=3, label="3m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

In [119]:
fig = px.line(df, y=_crypto_vol, title='Crypto Vol. Time Series', width = 1000, height = 400)

fig.update_xaxes(
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=3, label="3m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

### `Index`

In [120]:
df[_index_price + _index_vol].describe()

,S&P_500_Price,Nasdaq_100_Price,Berkshire_Price,Nasdaq_100_Vol.,Berkshire_Vol.
count,1243.000000,1243.000000,1243.000000,1.242000e+03,1243.000000
mean,3793.322542,12037.318101,404273.051488,2.238827e+08,2426.524537
std,632.321579,2887.069742,86369.903899,8.246992e+07,2660.497572
min,2237.400000,6904.980000,240000.000000,5.047000e+07,80.000000
25%,3190.640000,9298.730000,318984.500000,1.715275e+08,345.000000
50%,3932.690000,12381.170000,418349.000000,2.116200e+08,1510.000000
75%,4349.240000,14563.250000,471500.000000,2.559850e+08,3225.000000
max,4958.610000,17642.730000,589498.000000,9.825600e+08,13850.000000


In [121]:
fig = px.line(df, y=_index_price, title='Index Price Time Series', width = 1000, height = 400)

fig.update_xaxes(
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=3, label="3m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

In [122]:
fig = px.line(df, y=_index_vol, title='Index Vol. Time Series', width = 1000, height = 400)

fig.update_xaxes(
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=3, label="3m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

In [123]:
df['Nasdaq_100_Vol.'].interpolate(method = 'linear', inplace=True)

### `Big Techs`

In [124]:
df[_techs_price + _techs_vol].describe()

,Apple_Price,Tesla_Price,Microsoft_Price,Google_Price,Nvidia_Price,Netflix_Price,Amazon_Price,Meta_Price,Apple_Vol.,Tesla_Vol.,Microsoft_Vol.,Google_Vol.,Nvidia_Vol.,Netflix_Vol.,Amazon_Vol.,Meta_Vol.
count,1243.000000,1243.000000,1243.000000,1243.000000,1243.000000,1243.000000,1243.000000,1243.000000,1.243000e+03,1.243000e+03,1.243000e+03,1.243000e+03,1.243000e+03,1.243000e+03,1.243000e+03,1.243000e+03
mean,125.566533,173.590442,241.240072,99.670829,187.285841,404.839541,128.683234,239.728134,9.981845e+07,1.341790e+08,2.920801e+07,3.351599e+07,4.560298e+07,7.057401e+06,7.413005e+07,2.325851e+07
std,46.114122,106.688094,73.429838,29.766893,134.679941,114.989473,30.808631,71.015427,5.208192e+07,8.509129e+07,1.236710e+07,1.507215e+07,1.869107e+07,6.384187e+06,3.245753e+07,1.555486e+07
min,42.360000,11.930000,105.250000,51.940000,33.450000,166.370000,79.410000,88.910000,2.404000e+07,2.940000e+07,8.990000e+06,9.310000e+06,9.790000e+06,1.140000e+06,1.763000e+07,5.470000e+06
25%,79.505000,53.335000,182.525000,71.185000,73.905000,323.010000,96.260000,183.355000,6.599000e+07,8.136000e+07,2.153500e+07,2.410500e+07,3.245000e+07,3.990000e+06,5.264500e+07,1.478500e+07
50%,136.760000,202.070000,245.380000,101.240000,151.590000,384.150000,128.730000,224.430000,8.749000e+07,1.095200e+08,2.610000e+07,2.973000e+07,4.279000e+07,5.610000e+06,6.520000e+07,1.934000e+07
75%,162.915000,251.470000,297.120000,127.205000,242.140000,495.365000,158.110000,301.650000,1.170750e+08,1.578300e+08,3.317500e+07,3.735500e+07,5.511500e+07,7.910000e+06,8.674500e+07,2.711500e+07
max,198.110000,409.970000,411.220000,153.510000,661.600000,691.690000,186.570000,474.990000,4.268800e+08,9.140800e+08,9.707000e+07,1.331800e+08,1.534600e+08,1.333900e+08,3.113500e+08,2.304100e+08


In [125]:
fig = px.line(df, y=_techs_price, title='Big Techs Price Time Series', width = 1000, height = 400)

fig.update_xaxes(
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=3, label="3m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

In [126]:
fig = px.line(df, y=_techs_vol, title='Big Techs Vol. Time Series', width = 1000, height = 400)

fig.update_xaxes(
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=3, label="3m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1243 entries, 2019-02-04 to 2024-02-02
Data columns (total 36 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Natural_Gas_Price  1243 non-null   float64
 1   Natural_Gas_Vol.   1243 non-null   float64
 2   Crude_oil_Price    1243 non-null   float64
 3   Crude_oil_Vol.     1243 non-null   float64
 4   Copper_Price       1243 non-null   float64
 5   Copper_Vol.        1243 non-null   float64
 6   Bitcoin_Price      1243 non-null   float64
 7   Bitcoin_Vol.       1243 non-null   float64
 8   Platinum_Price     1243 non-null   float64
 9   Ethereum_Price     1243 non-null   float64
 10  Ethereum_Vol.      1243 non-null   float64
 11  S&P_500_Price      1243 non-null   float64
 12  Nasdaq_100_Price   1243 non-null   float64
 13  Nasdaq_100_Vol.    1243 non-null   float64
 14  Apple_Price        1243 non-null   float64
 15  Apple_Vol.         1243 non-null   float64
 16  Tesla_

# Note:
The data should be clean now and ready for next step: EDA. I plan to use this dataset for more analysis like stock price prediction using parametric regression approach, neural network, or LSTM network... It would also be a good exercise to apply risk data analysis like volatility and value-at-risk calculation. Hopefully I could enrich the content of this notebook later!